# Day 21: 优秀 Agent 设计模式

从 Kaggle Capstone 获奖项目中学习的设计模式

## 1. Multi-Agent 编排模式

**核心思想**: 将复杂任务拆分给多个专职 Agent，每个 Agent 只做一件事。

In [1]:
from dataclasses import dataclass
from typing import Any
from abc import ABC, abstractmethod


# 定义 Agent 基类
class BaseAgent(ABC):
    """所有 Agent 的基类"""
    
    @abstractmethod
    def process(self, input_data: Any) -> Any:
        pass


# 专职 Agent 示例
class RetrieverAgent(BaseAgent):
    """只负责获取数据"""
    
    def process(self, query: str) -> list[dict]:
        # 模拟获取新闻
        return [
            {"title": "AI 突破", "source": "TechNews", "content": "..."},
            {"title": "新模型发布", "source": "AIDaily", "content": "..."},
        ]


class VerifierAgent(BaseAgent):
    """只负责验证内容真实性"""
    
    def process(self, articles: list[dict]) -> list[dict]:
        # 模拟验证 - 检查是否有可信来源
        verified = []
        for article in articles:
            if article.get("source"):  # 有来源才通过
                article["verified"] = True
                verified.append(article)
        return verified


class GeneratorAgent(BaseAgent):
    """只负责生成摘要"""
    
    def process(self, articles: list[dict]) -> str:
        # 模拟生成简报
        summaries = [f"• {a['title']} (来源: {a['source']})" for a in articles]
        return "\n".join(summaries)


# 编排器 - 协调多个 Agent
class NewsOrchestrator:
    """Multi-Agent 编排器"""
    
    def __init__(self):
        self.retriever = RetrieverAgent()
        self.verifier = VerifierAgent()
        self.generator = GeneratorAgent()
    
    def run(self, query: str) -> str:
        # 流水线执行
        print(f"1. 检索: {query}")
        articles = self.retriever.process(query)
        print(f"   获取到 {len(articles)} 篇文章")
        
        print("2. 验证来源...")
        verified = self.verifier.process(articles)
        print(f"   {len(verified)} 篇通过验证")
        
        print("3. 生成简报...")
        result = self.generator.process(verified)
        
        return result


# 运行
orchestrator = NewsOrchestrator()
newsletter = orchestrator.run("AI 最新进展")
print("\n📰 今日简报:")
print(newsletter)

1. 检索: AI 最新进展
   获取到 2 篇文章
2. 验证来源...
   2 篇通过验证
3. 生成简报...

📰 今日简报:
• AI 突破 (来源: TechNews)
• 新模型发布 (来源: AIDaily)


## 2. Self-Correction Loop 模式

**核心思想**: 用 LLM 评估输出质量，不达标就重新生成。

In [2]:
@dataclass
class EvaluationResult:
    passed: bool
    score: float
    feedback: str


class QualityEvaluator:
    """质量评估器 - 模拟 LLM 评估"""
    
    def __init__(self, threshold: float = 0.7):
        self.threshold = threshold
    
    def evaluate(self, content: str) -> EvaluationResult:
        # 模拟评估逻辑
        checks = {
            "has_content": len(content) > 10,
            "has_source": "来源" in content or "source" in content.lower(),
            "no_placeholder": "..." not in content and "TODO" not in content,
        }
        
        score = sum(checks.values()) / len(checks)
        
        if score >= self.threshold:
            return EvaluationResult(True, score, "质量达标")
        else:
            failed = [k for k, v in checks.items() if not v]
            return EvaluationResult(False, score, f"需改进: {failed}")


class SelfCorrectingAgent:
    """自纠错 Agent"""
    
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries
        self.evaluator = QualityEvaluator()
        self.attempt = 0
    
    def generate(self, task: str, feedback: str = "") -> str:
        """模拟生成 - 根据反馈改进"""
        self.attempt += 1
        
        if self.attempt == 1:
            # 第一次：低质量输出
            return "AI 有新进展... TODO"
        elif self.attempt == 2:
            # 第二次：改进但仍有问题
            return "AI 有新进展，详情待补充..."
        else:
            # 第三次：高质量输出
            return "AI 最新进展：GPT-5 发布 (来源: OpenAI 官方博客)"
    
    def run(self, task: str) -> str:
        feedback = ""
        
        for attempt in range(1, self.max_retries + 1):
            print(f"\n🔄 尝试 {attempt}/{self.max_retries}")
            
            result = self.generate(task, feedback)
            print(f"   生成: {result}")
            
            evaluation = self.evaluator.evaluate(result)
            print(f"   评分: {evaluation.score:.1%} - {evaluation.feedback}")
            
            if evaluation.passed:
                print("   ✅ 通过!")
                return result
            
            feedback = evaluation.feedback
            print(f"   ❌ 未通过，继续改进...")
        
        raise Exception("无法达到质量标准")


# 运行自纠错流程
agent = SelfCorrectingAgent()
final_result = agent.run("写一条 AI 新闻")
print(f"\n📝 最终结果: {final_result}")


🔄 尝试 1/3
   生成: AI 有新进展... TODO
   评分: 33.3% - 需改进: ['has_source', 'no_placeholder']
   ❌ 未通过，继续改进...

🔄 尝试 2/3
   生成: AI 有新进展，详情待补充...
   评分: 33.3% - 需改进: ['has_source', 'no_placeholder']
   ❌ 未通过，继续改进...

🔄 尝试 3/3
   生成: AI 最新进展：GPT-5 发布 (来源: OpenAI 官方博客)
   评分: 100.0% - 质量达标
   ✅ 通过!

📝 最终结果: AI 最新进展：GPT-5 发布 (来源: OpenAI 官方博客)


## 3. Citation Verification 模式

**核心思想**: 每个声明都必须有可追溯的来源，实现零幻觉。

In [3]:
@dataclass
class Citation:
    """引用来源"""
    source: str
    url: str
    accessed_at: str


@dataclass 
class VerifiedClaim:
    """带引用的声明"""
    claim: str
    citation: Citation
    confidence: float


class CitationVerifier:
    """引用验证器"""
    
    def verify_and_cite(self, claims: list[str], sources: list[dict]) -> list[VerifiedClaim]:
        """为每个声明找到并验证引用"""
        verified_claims = []
        
        for claim in claims:
            # 模拟查找匹配的来源
            matching_source = self._find_source(claim, sources)
            
            if matching_source:
                verified_claims.append(VerifiedClaim(
                    claim=claim,
                    citation=Citation(
                        source=matching_source["name"],
                        url=matching_source["url"],
                        accessed_at="2025-12-29"
                    ),
                    confidence=0.95
                ))
            else:
                print(f"⚠️ 无法验证: {claim}")
        
        return verified_claims
    
    def _find_source(self, claim: str, sources: list[dict]) -> dict | None:
        """模拟语义匹配查找来源"""
        for source in sources:
            # 简化：关键词匹配
            if any(kw in claim.lower() for kw in source.get("keywords", [])):
                return source
        return None


# 示例：验证声明
claims = [
    "GPT-5 将于 2025 年发布",
    "Gemini 2.0 支持多模态",
    "某公司股价上涨 100%",  # 无来源
]

sources = [
    {"name": "OpenAI Blog", "url": "https://openai.com/blog", "keywords": ["gpt", "openai"]},
    {"name": "Google AI Blog", "url": "https://ai.google/blog", "keywords": ["gemini", "google"]},
]

verifier = CitationVerifier()
verified = verifier.verify_and_cite(claims, sources)

print("\n✅ 已验证的声明:")
for vc in verified:
    print(f"  • {vc.claim}")
    print(f"    📎 来源: {vc.citation.source} ({vc.citation.url})")
    print(f"    🎯 置信度: {vc.confidence:.0%}")

⚠️ 无法验证: 某公司股价上涨 100%

✅ 已验证的声明:
  • GPT-5 将于 2025 年发布
    📎 来源: OpenAI Blog (https://openai.com/blog)
    🎯 置信度: 95%
  • Gemini 2.0 支持多模态
    📎 来源: Google AI Blog (https://ai.google/blog)
    🎯 置信度: 95%


## 4. User Feedback Loop 模式

**核心思想**: 从用户反馈中学习，持续改进 Agent 行为。

In [4]:
from collections import defaultdict


@dataclass
class UserFeedback:
    """用户反馈"""
    content_id: str
    rating: int  # 1-5
    tags: list[str]  # ["relevant", "accurate", "too_long", etc.]


class FeedbackLearner:
    """反馈学习器"""
    
    def __init__(self):
        self.feedback_history: list[UserFeedback] = []
        self.tag_stats = defaultdict(int)
        self.preferences = {}
    
    def record_feedback(self, feedback: UserFeedback):
        """记录反馈"""
        self.feedback_history.append(feedback)
        for tag in feedback.tags:
            self.tag_stats[tag] += 1
        self._update_preferences()
    
    def _update_preferences(self):
        """根据反馈更新偏好"""
        total = len(self.feedback_history)
        if total == 0:
            return
        
        # 计算各标签比例
        self.preferences = {
            "avg_rating": sum(f.rating for f in self.feedback_history) / total,
            "wants_shorter": self.tag_stats["too_long"] / total > 0.3,
            "wants_more_detail": self.tag_stats["too_brief"] / total > 0.3,
            "prefers_technical": self.tag_stats["technical"] / total > 0.5,
        }
    
    def get_generation_hints(self) -> dict:
        """获取生成提示，用于调整 Agent 行为"""
        hints = {}
        
        if self.preferences.get("wants_shorter"):
            hints["max_length"] = "short"
        elif self.preferences.get("wants_more_detail"):
            hints["max_length"] = "detailed"
        
        if self.preferences.get("prefers_technical"):
            hints["style"] = "technical"
        else:
            hints["style"] = "casual"
        
        return hints


# 模拟用户反馈循环
learner = FeedbackLearner()

# 模拟收集反馈
feedbacks = [
    UserFeedback("news_1", 3, ["too_long", "relevant"]),
    UserFeedback("news_2", 4, ["technical", "accurate"]),
    UserFeedback("news_3", 2, ["too_long", "irrelevant"]),
    UserFeedback("news_4", 5, ["technical", "relevant", "accurate"]),
    UserFeedback("news_5", 4, ["technical"]),
]

print("📊 收集用户反馈...")
for fb in feedbacks:
    learner.record_feedback(fb)
    print(f"  {fb.content_id}: ⭐{fb.rating} - {fb.tags}")

print(f"\n📈 学习到的偏好:")
for k, v in learner.preferences.items():
    print(f"  {k}: {v}")

print(f"\n🎯 生成提示 (用于调整 Agent):")
hints = learner.get_generation_hints()
for k, v in hints.items():
    print(f"  {k}: {v}")

📊 收集用户反馈...
  news_1: ⭐3 - ['too_long', 'relevant']
  news_2: ⭐4 - ['technical', 'accurate']
  news_3: ⭐2 - ['too_long', 'irrelevant']
  news_4: ⭐5 - ['technical', 'relevant', 'accurate']
  news_5: ⭐4 - ['technical']

📈 学习到的偏好:
  avg_rating: 3.6
  wants_shorter: True
  wants_more_detail: False
  prefers_technical: True

🎯 生成提示 (用于调整 Agent):
  max_length: short
  style: technical


## 5. 完整示例：NewsPulse 风格的 Agent

整合以上所有模式

In [5]:
class NewsPulseStyleAgent:
    """整合所有设计模式的 Agent"""
    
    def __init__(self):
        # Multi-Agent
        self.retriever = RetrieverAgent()
        self.verifier = VerifierAgent()
        self.generator = GeneratorAgent()
        
        # Quality Control
        self.evaluator = QualityEvaluator(threshold=0.6)
        
        # Citation
        self.citation_verifier = CitationVerifier()
        
        # Feedback Learning
        self.feedback_learner = FeedbackLearner()
    
    def generate_newsletter(self, query: str, max_retries: int = 3) -> dict:
        """生成新闻简报 - 完整流程"""
        
        print("="*50)
        print(f"🚀 生成简报: {query}")
        print("="*50)
        
        # 1. 获取用户偏好
        hints = self.feedback_learner.get_generation_hints()
        print(f"\n📋 用户偏好: {hints}")
        
        # 2. Multi-Agent 流水线
        print("\n🔍 Step 1: 检索新闻...")
        articles = self.retriever.process(query)
        
        print("\n✅ Step 2: 验证来源...")
        verified_articles = self.verifier.process(articles)
        
        # 3. Self-Correction Loop
        print("\n📝 Step 3: 生成简报 (with self-correction)...")
        for attempt in range(1, max_retries + 1):
            content = self.generator.process(verified_articles)
            
            # 评估质量
            eval_result = self.evaluator.evaluate(content)
            print(f"   尝试 {attempt}: 评分 {eval_result.score:.0%}")
            
            if eval_result.passed:
                break
        
        print("\n🎉 生成完成!")
        
        return {
            "content": content,
            "sources": [a["source"] for a in verified_articles],
            "quality_score": eval_result.score,
        }
    
    def collect_feedback(self, content_id: str, rating: int, tags: list[str]):
        """收集用户反馈"""
        feedback = UserFeedback(content_id, rating, tags)
        self.feedback_learner.record_feedback(feedback)
        print(f"✅ 反馈已记录: {content_id}")


# 运行完整示例
agent = NewsPulseStyleAgent()

# 模拟一些历史反馈
agent.collect_feedback("old_1", 4, ["technical"])
agent.collect_feedback("old_2", 3, ["too_long"])

# 生成新简报
result = agent.generate_newsletter("AI 最新动态")

print("\n" + "="*50)
print("📰 最终输出:")
print("="*50)
print(f"内容: {result['content']}")
print(f"来源: {result['sources']}")
print(f"质量: {result['quality_score']:.0%}")

✅ 反馈已记录: old_1
✅ 反馈已记录: old_2
🚀 生成简报: AI 最新动态

📋 用户偏好: {'max_length': 'short', 'style': 'casual'}

🔍 Step 1: 检索新闻...

✅ Step 2: 验证来源...

📝 Step 3: 生成简报 (with self-correction)...
   尝试 1: 评分 100%

🎉 生成完成!

📰 最终输出:
内容: • AI 突破 (来源: TechNews)
• 新模型发布 (来源: AIDaily)
来源: ['TechNews', 'AIDaily']
质量: 100%


## 设计检查清单

在构建 Agent 时，检查以下要点：

- [ ] **问题定义** - 是否足够具体？
- [ ] **Multi-Agent** - 是否需要拆分成多个专职 Agent？
- [ ] **Self-Correction** - 有没有质量评估和自纠错机制？
- [ ] **Citation** - 输出是否可追溯（有引用来源）？
- [ ] **Feedback Loop** - 是否有用户反馈的学习机制？
- [ ] **MCP 标准化** - 工具能力是否通过 MCP 标准化？